# **Generate Class Image for Dreambooth**


In [ ]:
import os
from tqdm import tqdm

import torch
from diffusers import DiffusionPipeline

output_path = "/data/noah/dataset/dreambooth_AD/class/"
model_id = "/data/noah/ckpt/pretrain_ckpt/StableDiffusion/rv/"
device_id = "cuda:2"
prompt = "a photo of the cars in motion, best quality, extremely detailed, clearness, naturalness, film grain, crystal clear, photo with color, actuality"
negative_prompt = "cartoon, anime, painting, disfigured, immature, blur, picture, ugly, 3D, render, semi-realistic, drawing, poorly drawn, bad anatomy, wrong anatomy, gray scale, worst quality, low quality, sketch"
num_images = 1000

pipe = DiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16, use_safetensors=True).to(device_id)

for _ in tqdm(range(num_images)):
    image = pipe(prompt=prompt, negative_prompt=negative_prompt, guidance_scale=9.0, num_inference_steps=50).images[0]
    image.save(os.path.join(output_path, "{}.jpg".format(_)))
# image.save()

# **Get Image Dataset**


In [ ]:
import os
import shutil

src = "/data/noah/dataset/batch_3"
dst = "/data/noah/dataset/AD_REIMAGINE/test"
max_copy = 1000

cnt = 0

for path, dirs, files in os.walk(src):
    for f in files:
        if os.path.splitext(f)[-1] in [".jpg", ".png"]:
            shutil.copy(os.path.join(path, f), os.path.join(dst, f))

        if cnt >= max_copy:
            break

    if cnt >= max_copy:
        break

In [1]:
import csv
from tqdm import tqdm

from PIL import Image
import torch
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode
from diffusers.blip.models.blip import blip_decoder


def load_demo_image(image_path, image_size, device):
    raw_image = Image.open(image_path).convert("RGB")

    w, h = raw_image.size
    # display(raw_image.resize((w//5,h//5)))

    transform = transforms.Compose(
        [
            transforms.Resize((image_size, image_size), interpolation=InterpolationMode.BICUBIC),
            transforms.CenterCrop(image_size),
            transforms.ToTensor(),
            transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711)),
        ]
    )
    image = transform(raw_image).unsqueeze(0).to(device)
    return image


image_size = 512
image_dir = "/data/noah/dataset/disney"
model_path = "/data/noah/ckpt/pretrain_ckpt/BLIP/model_large_caption.pth"
device = "cuda:2"
torch.cuda.set_device(torch.device(device))

model = blip_decoder(pretrained=model_path, image_size=image_size, vit="large")
model.eval()
model = model.to(device)

file_name = "/data/noah/dataset/disney/metadata.csv"
data = []

for img_name in tqdm(os.listdir(image_dir)):
    # if os.path.splitext(img_name)[-1] not in [".jpg", ".png"]:
    #     continue

    image_path = os.path.join(image_dir, img_name)

    image = load_demo_image(image_path=image_path, image_size=image_size, device=device)

    with torch.no_grad():
        # beam search
        # caption = model.generate(image, sample=False, num_beams=3, max_length=40, min_length=5)
        # nucleus sampling
        caption = model.generate(image, sample=True, top_p=0.9, max_length=20, min_length=5)

        data.append({"file_name": image_path, "text": caption[0]})

with open(file_name, mode="w", newline="") as file:
    writer = csv.DictWriter(file, fieldnames=["image_path", "text"])

    writer.writeheader()

    for row in data:
        writer.writerow(row)

print(f"{file_name} 파일이 생성되었습니다.")

KeyboardInterrupt: 

# **Get Models**


In [ ]:
import huggingface_hub

huggingface_hub.snapshot_download(
    repo_id="runwayml/stable-diffusion-v1-5",
    repo_type="model",
    local_dir="/data/noah/ckpt/pretrain_ckpt/StableDiffusion/sd",
    local_dir_use_symlinks=False,
)

In [ ]:
import cv2
import os

path = "/data/noah/inference/reimagine/input"
out_path = "/data/noah/inference/reimagine/input_2"

if not os.path.isdir(out_path):
    os.mkdir(out_path)

for _ in os.listdir(path):
    img_path = os.path.join(path, _)
    img = cv2.imread(img_path)

    img = cv2.resize(img, (768, 768))

    cv2.imwrite(os.path.join(out_path, _), img)

In [ ]:
import os
import csv
import json
from tqdm import tqdm
import shutil

num = 2500
meta_path = "/data/noah/dataset/AD_/train/metadata.csv"
src = "/data/noah/dataset/AD_/train/"
dst = "/data/noah/dataset/AD/train/"

data = []

# CSV 파일을 읽기 모드로 엽니다.
with open(meta_path, "r", newline="") as csvfile:
    # CSV 파일을 읽기 위한 reader 객체를 생성합니다.
    csv_reader = csv.reader(csvfile)

    # 각 행을 반복하면서 데이터를 처리합니다.
    for idx, row in enumerate(csv_reader):
        # 각 행의 데이터는 리스트 형태로 반환됩니다.
        # 여기에서 데이터를 사용하거나 출력합니다.
        if not idx:
            continue

        data.append({"file_name": row[0], "text": row[1]})

        if idx >= num:
            break

with open(os.path.join(dst, "metadata.csv"), mode="w", newline="") as file:
    writer = csv.DictWriter(file, fieldnames=["file_name", "text"])

    writer.writeheader()

    for row in tqdm(data):
        writer.writerow(row)
        shutil.copy(os.path.join(src, row["file_name"]), os.path.join(dst, row["file_name"]))

In [ ]:
import os
import csv
import json
from tqdm import tqdm
import shutil

meta_path = "/data/noah/inference/data_filtering/data_filtering.csv"
data = []

# CSV 파일을 읽기 모드로 엽니다.
with open(meta_path, "r", newline="") as csvfile:
    # CSV 파일을 읽기 위한 reader 객체를 생성합니다.
    csv_reader = csv.reader(csvfile)

    # 각 행을 반복하면서 데이터를 처리합니다.
    for idx, row in enumerate(csv_reader):
        data.append({"file_name": row[0], "score": row[1]})

In [ ]:
base_path = "/data/noah/inference/data_filtering/synthetic"
sorted_data = sorted(data, key=lambda x: x["score"])
sorted_data

In [ ]:
import matplotlib.pyplot as plt

target_len = 20
top_data = sorted_data[:target_len]
top_images = []

for t in top_data:
    img_path = os.path.join(base_path, t["file_name"])
    img = Image.open(img_path).convert("RGB")
    top_images.append(img)

plt.figure(figsize=(50, 35))

for i in range(target_len):
    plt.subplot(4, 5, i + 1)
    plt.imshow(top_images[i])
    plt.title(top_dis[i]["syn_name"])
    plt.axis("off")

plt.show()

# **Generate ControlNet Dataset**


In [32]:
import cv2
import os
import csv
import json
import shutil
import numpy as np
from tqdm import tqdm
import torch
from PIL import Image
from mobile_sam import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
from mobile_sam import SamAutomaticMaskGenerator
import random

device = "cuda:3"
torch.cuda.set_device(device)

src = "/data/noah/dataset/AD_SD/train"
meta_path = "/data/noah/dataset/AD_SD/train/metadata.csv"

dst = "/data/noah/dataset/AD_CON"
ann_path = "/data/noah/dataset/AD_CON/train.jsonl"

model_type = "vit_t"
ckpt_path = "/data/noah/ckpt/pretrain_ckpt/M_SAM/mobile_sam.pt"
mobile_sam = sam_model_registry[model_type](checkpoint=ckpt_path)
mobile_sam.to(device=device)
mobile_sam.eval()
mask_generator = SamAutomaticMaskGenerator(mobile_sam)

data = []

with open(meta_path, "r", newline="") as csvfile:
    # CSV 파일을 읽기 위한 reader 객체를 생성합니다.
    csv_reader = csv.reader(csvfile)

    # 각 행을 반복하면서 데이터를 처리합니다.
    for idx, row in tqdm(enumerate(csv_reader)):
        # 각 행의 데이터는 리스트 형태로 반환됩니다.
        # 여기에서 데이터를 사용하거나 출력합니다.
        if not idx:
            continue

        data.append(
            {"text": row[1], "image": "images/" + row[0], "conditioning_image": "conditioning_images/" + row[0]}
        )
        shutil.copy(os.path.join(src, row[0]), os.path.join(dst + "/images", row[0]))
        img = Image.open(os.path.join(dst + "/images", row[0])).convert("RGB")
        masks = mask_generator.generate(np.array(img))

        seg_map = np.zeros((img.height, img.width, 3))
        color_map = []

        for mask in masks:
            while True:
                color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

                if color not in color_map:
                    color_map.append(color)
                    break

            seg_map[mask["segmentation"], :] = color

        seg_map = Image.fromarray(seg_map.astype("uint8"))
        seg_map.save(os.path.join(dst + "/conditioning_images", row[0]))

with open(os.path.join(dst, "train.jsonl"), encoding="utf-8", mode="w") as f:
    for i in data:
        f.write(json.dumps(i) + "\n")

14it [00:59,  4.24s/it]


KeyboardInterrupt: 

In [1]:
import cv2
import os
import csv
import json
import shutil
import numpy as np
from tqdm import tqdm
import torch
from PIL import Image
from controlnet_aux import LeresDetector
import random

device = "cuda:3"
torch.cuda.set_device(device)

src = "/data/noah/dataset/AD_SD/train"
meta_path = "/data/noah/dataset/AD_SD/train/metadata.csv"

dst = "/data/noah/dataset/AD_CON"
ann_path = "/data/noah/dataset/AD_CON/train.jsonl"


data = []
leres = LeresDetector.from_pretrained("lllyasviel/Annotators")

with open(meta_path, "r", newline="") as csvfile:
    # CSV 파일을 읽기 위한 reader 객체를 생성합니다.
    csv_reader = csv.reader(csvfile)

    # 각 행을 반복하면서 데이터를 처리합니다.
    for idx, row in tqdm(enumerate(csv_reader)):
        # 각 행의 데이터는 리스트 형태로 반환됩니다.
        # 여기에서 데이터를 사용하거나 출력합니다.
        if not idx:
            continue

        data.append(
            {"text": row[1], "image": "images/" + row[0], "conditioning_image": "conditioning_images/" + row[0]}
        )
        shutil.copy(os.path.join(src, row[0]), os.path.join(dst + "/images", row[0]))
        img = Image.open(os.path.join(dst + "/images", row[0])).convert("RGB")
        leres_img = leres(np.array(img))
        leres_img.save(os.path.join(dst + "/conditioning_images", row[0]))

with open(os.path.join(dst, "train.jsonl"), encoding="utf-8", mode="w") as f:
    for i in data:
        f.write(json.dumps(i) + "\n")

res101.pth:   0%|          | 0.00/531M [00:00<?, ?B/s]

latest_net_G.pth:   0%|          | 0.00/318M [00:00<?, ?B/s]

57it [02:21,  2.48s/it]


KeyboardInterrupt: 

In [24]:
a = np.zeros((10, 20))
a.shape
b = Image.fromarray(a).convert("RGB")
display(b)

In [ ]:
import os
import cv2

path = "/data/noah/inference/reimagine/input"
out = "/data/noah/inference/reimagine/output_i"

for _ in os.listdir(path):
    img = cv2.imread(os.path.join(path, _))

    img = cv2.resize(img, (768, 768))

    cv2.imwrite(os.path.join(out, _), img)

In [ ]:
import requests
from PIL import Image
from io import BytesIO

from controlnet_aux import CannyDetector, SamDetector

# load image
img = Image.open("/data/noah/dataset/AD/train/1652247588842_FR-View-CMR-Wide.png").convert("RGB").resize((512, 512))

sam = SamDetector.from_pretrained("ybelkada/segment-anything", subfolder="checkpoints")

processed_image_2 = sam(img)

display(img)
display(processed_image_2)

# **Preprocess Sequence Datasset**


In [ ]:
import av
import numpy as np
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForCausalLM

device = "cuda:3"
torch.cuda.set_device(device)

processor = AutoProcessor.from_pretrained("/data/noah/ckpt/pretrain_ckpt/GIT")
model = AutoModelForCausalLM.from_pretrained("/data/noah/ckpt/pretrain_ckpt/GIT").to(device)

# set seed for reproducability


def read_video_pyav(container, indices):
    """
    Decode the video with PyAV decoder.
    Args:
        container (`av.container.input.InputContainer`): PyAV container.
        indices (`List[int]`): List of frame indices to decode.
    Returns:
        result (np.ndarray): np array of decoded frames of shape (num_frames, height, width, 3).
    """
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])


def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    """
    Sample a given number of frame indices from the video.
    Args:
        clip_len (`int`): Total number of frames to sample.
        frame_sample_rate (`int`): Sample every n-th frame.
        seg_len (`int`): Maximum allowed index of sample's last frame.
    Returns:
        indices (`List[int]`): List of sampled frame indices
    """
    converted_len = int(clip_len * frame_sample_rate)
    end_idx = np.random.randint(converted_len, seg_len)
    start_idx = end_idx - converted_len
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices


# load video
file_path = "/data/noah/inference/incabin_sample/reference_vid/sample_2.mp4"
container = av.open(file_path)

# sample frames
num_frames = model.config.num_image_with_embedding
indices = sample_frame_indices(clip_len=num_frames, frame_sample_rate=4, seg_len=container.streams.video[0].frames)
frames = read_video_pyav(container, indices)

pixel_values = processor(images=list(frames), return_tensors="pt").pixel_values
generated_ids = model.generate(pixel_values=pixel_values.to(device), max_length=50)

print("Generated caption:", processor.batch_decode(generated_ids, skip_special_tokens=True))

In [ ]:
import os

dataset_base_path = "/data/noah/dataset/AD_SEQ"
folder_paths = []

for root, dirs, files in os.walk(dataset_base_path):
    for file in files:
        if root not in folder_paths and file[-4:] in [".jpg", ".png", ".jpeg"]:
            folder_paths.append(root)

folder_paths

In [ ]:
import os
import numpy as np
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForCausalLM

device = "cuda:2"
torch.cuda.set_device(device)

processor = AutoProcessor.from_pretrained("/data/noah/ckpt/pretrain_ckpt/GIT")
model = AutoModelForCausalLM.from_pretrained("/data/noah/ckpt/pretrain_ckpt/GIT").to(device)

# set seed for reproducability


def read_video_pyav(container, indices):
    """
    Decode the video with PyAV decoder.
    Args:
        container (`av.container.input.InputContainer`): PyAV container.
        indices (`List[int]`): List of frame indices to decode.
    Returns:
        result (np.ndarray): np array of decoded frames of shape (num_frames, height, width, 3).
    """
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])


def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    """
    Sample a given number of frame indices from the video.
    Args:
        clip_len (`int`): Total number of frames to sample.
        frame_sample_rate (`int`): Sample every n-th frame.
        seg_len (`int`): Maximum allowed index of sample's last frame.
    Returns:
        indices (`List[int]`): List of sampled frame indices
    """
    converted_len = int(clip_len * frame_sample_rate)
    end_idx = np.random.randint(converted_len, seg_len)
    start_idx = end_idx - converted_len
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices


# load video
folder_path = "/data/noah/dataset/AD_SEQ"
captions = []
folder_paths = []

for root, folders, files in os.walk(folder_path):
    frames = []
    for file in files:
        if file.endswith(".png"):
            if root not in folder_paths:
                folder_paths.append(root)
            frame = Image.open(os.path.join(root, file)).convert("RGB")
            frames.append(frame)

    ###
    if not len(frames):
        continue
    seg_len = len(frames)
    num_frames = model.config.num_image_with_embedding
    indices = sorted(
        sample_frame_indices(clip_len=num_frames, frame_sample_rate=seg_len // num_frames, seg_len=seg_len)
    )
    selected_frames = [frames[idx] for idx in indices]
    pixel_values = processor(images=selected_frames, return_tensors="pt").pixel_values
    generated_ids = model.generate(pixel_values=pixel_values.to(device), max_length=50)
    captions.append(processor.batch_decode(generated_ids, skip_special_tokens=True)[0])

In [ ]:
import csv

out_path = "/data/noah/dataset/AD_SEQ/metadata.csv"

with open(out_path, mode="w", newline="") as file:
    writer = csv.DictWriter(file, fieldnames=["folder", "caption"])

    writer.writeheader()

    for row in zip(folder_paths):
        writer.writerow({"folder": row})

# Extract Videos from Source Video


In [6]:
import cv2
import os
from diffusers.utils import export_to_video
from PIL import Image as PILImage

videos_path = "/data/noah/inference/simulation/video_src"
out_video_path = "/data/noah/inference/simulation/video"


def extract_video(video_path, extact_frame=16, sample_step=8):
    cap = cv2.VideoCapture(video_path)
    video_name = os.path.basename(video_path)
    frames = []
    cnt = 0
    sample_cnt = 0

    while True:
        ret, frame = cap.read()

        if not ret:
            break

        sample_cnt += 1

        if sample_cnt % 4 != 0:
            continue

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = PILImage.fromarray(frame)
        frames.append(frame)

        if len(frames) == 16:
            out_path = os.path.join(out_video_path, "{}_{}.mp4".format(video_name[:-4], cnt))
            export_to_video(frames, out_path, fps=5)
            cnt += 1
            frames = []

    cap.release()


for video_name in os.listdir(videos_path):
    video_path = os.path.join(videos_path, video_name)
    extract_video(video_path)

# Quration for Sequence Dataset
